<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/exemplo_programacao_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solução de problemas usando programação linear

https://coin-or.github.io/pulp

Optimization with PuLP

PuLP is an linear and mixed integer programming modeler written in Python.

## Alocação de Tarefas com Programação Linear em Python

### Entendendo o Problema
Imagine uma empresa com 3 funcionários (A, B, C) e 4 tarefas (T1, T2, T3, T4). Cada funcionário possui uma eficiência diferente para cada tarefa, resultando em tempos de execução distintos. O objetivo é atribuir cada tarefa a um único funcionário de modo que o tempo total de execução seja minimizado.

### Modelando o Problema
* **Variáveis de decisão:** x[i][j] = 1 se a tarefa i for atribuída ao funcionário j, 0 caso contrário.
* **Função objetivo:** Minimizar a soma do produto entre o tempo de execução da tarefa e a variável de decisão.
* **Restrições:**
    * Cada tarefa deve ser atribuída a apenas um funcionário.
    * Cada funcionário não pode receber mais tarefas do que sua capacidade.


In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 40.0 MB/s eta 0:00:00


In [ ]:
from pulp import *

# 3a tentativa

In [17]:
# Dados do problema
tempo_execucao = [[3, 2, 4],  # Tempo de execução da tarefa i pelo funcionário j
                  [1, 5, 3],
                  [2, 4, 1],
                  [4, 3, 2]]

num_tarefas = len(tempo_execucao)
num_funcionarios = len(tempo_execucao[0])

In [22]:
print("num_tarefas", num_tarefas)
print("num_funcionarios", num_funcionarios)
print("tempo_execucao das tarefas", tempo_execucao)

num_tarefas 4
num_funcionarios 3
tempo_execucao das tarefas [[3, 2, 4], [1, 5, 3], [2, 4, 1], [4, 3, 2]]


In [34]:
meus_funcionarios = {0:'A', 1:'B', 2:'C'}
minhas_tarefas = {0:'T1', 1:'T2', 2:'T3', 3:'T4'}

In [37]:
# Criar o problema de minimização
prob = LpProblem("Alocacao_Tarefas", LpMinimize)

# Criar as variáveis de decisão
x = LpVariable.dicts("Alocacao", [(i, j) for i in range(num_tarefas) for j in range(num_funcionarios)], 0, 1, LpBinary)

print("Variáveis de decisão: ")
for k, v in x.items():
    print(f"Tarefa: {minhas_tarefas[k[0]]}, Funcionário: {meus_funcionarios[k[1]]}, Tarefa alocada: {v}")

Variáveis de decisão: 
Tarefa: T1, Funcionário: A, Tarefa alocada: Alocacao_(0,_0)
Tarefa: T1, Funcionário: B, Tarefa alocada: Alocacao_(0,_1)
Tarefa: T1, Funcionário: C, Tarefa alocada: Alocacao_(0,_2)
Tarefa: T2, Funcionário: A, Tarefa alocada: Alocacao_(1,_0)
Tarefa: T2, Funcionário: B, Tarefa alocada: Alocacao_(1,_1)
Tarefa: T2, Funcionário: C, Tarefa alocada: Alocacao_(1,_2)
Tarefa: T3, Funcionário: A, Tarefa alocada: Alocacao_(2,_0)
Tarefa: T3, Funcionário: B, Tarefa alocada: Alocacao_(2,_1)
Tarefa: T3, Funcionário: C, Tarefa alocada: Alocacao_(2,_2)
Tarefa: T4, Funcionário: A, Tarefa alocada: Alocacao_(3,_0)
Tarefa: T4, Funcionário: B, Tarefa alocada: Alocacao_(3,_1)
Tarefa: T4, Funcionário: C, Tarefa alocada: Alocacao_(3,_2)


In [38]:
# Função objetivo: Minimizar o tempo total de execução
prob += lpSum(tempo_execucao[i][j] * x[(i, j)] for i in range(num_tarefas) for j in range(num_funcionarios))

# Restrições: Cada tarefa deve ser atribuída a apenas um funcionário
for i in range(num_tarefas):
    prob += lpSum(x[(i, j)] for j in range(num_funcionarios)) == 1

In [39]:
# Restrições: Capacidade de cada funcionário (exemplo: cada funcionário pode fazer no máximo 2 tarefas)
capacidade_funcionarios = [2, 2, 2]  # Ajustar conforme necessário
for j in range(num_funcionarios):
    prob += lpSum(x[(i, j)] for i in range(num_tarefas)) <= capacidade_funcionarios[j]

In [31]:
# Resolver o problema
prob.solve()

# Verificar o status da solução
status = LpStatus[prob.status]
print(f"Status da solução: {status}")

# Imprimir a solução, se for ótima
if status == "Optimal":
    for i in range(num_tarefas):
        for j in range(num_funcionarios):
            if x[(i, j)].varValue == 1:
                print(f"Tarefa {i+1} atribuída ao funcionário {j+1}")
else:
    print("Não foi encontrada uma solução ótima.")


Status da solução: Optimal
Tarefa 1 atribuída ao funcionário 2
Tarefa 2 atribuída ao funcionário 1
Tarefa 3 atribuída ao funcionário 3
Tarefa 4 atribuída ao funcionário 3


### Explicação do Código:

1. **Definição dos dados:** Definimos uma matriz `tempo_execucao` que contém o tempo de execução de cada tarefa por cada funcionário.
2. **Criação do problema:** Criamos um problema de minimização usando `LpProblem`.
3. **Variáveis de decisão:** Criamos variáveis binárias `x[i][j]` para representar se a tarefa i é atribuída ao funcionário j.
4. **Função objetivo:** Definimos a função objetivo como a soma do produto entre o tempo de execução e a variável de decisão.
5. **Restrições:**
    * **Cada tarefa uma vez:** Cada tarefa deve ser atribuída a exatamente um funcionário.
    * **Capacidade dos funcionários:** Cada funcionário não pode receber mais tarefas do que sua capacidade.
6. **Resolução:** Utilizamos o método `solve()` para encontrar a solução ótima.
7. **Impressão da solução:** Imprimimos quais tarefas foram atribuídas a cada funcionário.

**Observações:**

* **Flexibilidade:** Você pode ajustar os dados de entrada (tempo de execução, capacidade dos funcionários) para diferentes cenários.
* **Outras restrições:** É possível adicionar mais restrições, como preferências de funcionários por determinadas tarefas, prazos para as tarefas, etc.
* **Outras bibliotecas:** Além do PuLP, você pode utilizar outras bibliotecas como SciPy e OR-Tools para resolver problemas de programação linear.

**Extensões:**
* **Custos:** Você pode adicionar custos associados a cada atribuição.
* **Múltiplos recursos:** Considerar outros recursos além do tempo, como materiais ou máquinas.
* **Problemas mais complexos:** Resolver problemas de grande escala com milhares de variáveis e restrições.

Este exemplo demonstra como a programação linear pode ser utilizada para resolver problemas de alocação de recursos de forma eficiente e eficaz. Ao entender os princípios básicos e a implementação em Python, você poderá aplicar essa técnica em diversos outros problemas de otimização.